Genel Amaçlı Timer kullanılarak PWM sinyali elde etme   
PWM sinyali ile LED'in parlaklığı arttırılıyor, maksimum seviyeye geldiğinde tekrar en az seviye yapılıyor.

In [ ]:
#include "stm32f4xx.h"                  // Device header

unsigned int i=0;

void TIM3_IRQHandler(void){
    
    //Update Interrupt Flag 0 yapılıyor.
    TIM3->SR &= ~TIM_SR_UIF; 
    
    //Duty cycle için Capture Compare Register ayarlanıyor.
    //Once register içindeki değer temizleniyor.
    TIM3->CCR1 &= ~TIM_CCR1_CCR1;
    //i sayısı 0xFFFF ile maskelenerek Capture Compare Registera yazılıyor.
    TIM3->CCR1 |= (TIM_CCR1_CCR1&i);
    
    i=i+20;
    if(i>3200)
        i=0;

}


int main(){

    //TIM3 Kanal 3 için 3 pinden bir tanesi seçilebilir
    //Bu programda PA6 kullanılmıştır. 
    //Dolayısıyla A portuna clock sinyali verilmiştir.
    RCC->AHB1ENR |= RCC_AHB1ENR_GPIOAEN;
    
    //Timer 3'e clock sinyali verilmiştir.
    RCC->APB1ENR |= RCC_APB1ENR_TIM3EN;
    
    //Pinin modu alternatif mod olarak ayarlandı.
    GPIOA->MODER |= GPIO_MODER_MODE6_1;
    
    //ALTERNATIF MODE SELECTION
    //TIM 3 2 numaralı alternatif fonksiyon olduğu için iligli pine 0010 yazılıyor. 
    GPIOA->AFR[0] |= GPIO_AFRL_AFSEL6_1; 
    
    //PWM sinyali için Captura Compare işlemi aktif ediliyor.
    TIM3->CCER|= TIM_CCER_CC1E;
    
    //PWM sinyalinde Timer değerinin Capture Compare değerinden küçük olduğunda aktif
    //diğer durumda inaktif olmasını ayarlamak için TIM_CCMR yazmacının ilgili kanalına
    //110 yazılması
    TIM3->CCMR1 |= TIM_CCMR1_OC1M_2 | TIM_CCMR1_OC1M_1;
    
    //Update Interrupt Enable: sayma işlemi bittiğinde interrupt olması için
    TIM3->DIER |= TIM_DIER_UIE;
    
    //timer saymasını bitirdiğinde interrupt olması için
    NVIC_EnableIRQ(TIM3_IRQn);
    
    //Cihaz frekansının 100'e bölunmesi için
    TIM3->PSC = 99;
    //Auto Reload değeri, timer'ın maksimum değeri
    TIM3->ARR=3200U;
    
    //Duty Cycle için Capture Compare  yazmacının ayarlanması:
    TIM3->CCR1 |= TIM_CCR1_CCR1 & TIM3->ARR/4;
    
    //Timerı başlatmak için
    TIM3->CR1 |= TIM_CR1_CEN;
    
    while(1);
    
    return 0;
}
